In [24]:
from IPython.display import Image
from IPython.core.display import HTML 
#from __future__ import print_func  tion, division
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import json
from sklearn.model_selection import train_test_split
Image(url= "https://cdn-images-1.medium.com/max/1600/1*UkI9za9zTR-HL8uM15Wmzw.png")

pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 500)


In [25]:
# ------------- Hyperparameters ---------------------


# hyperparams

num_epoch = 100
total_series_length = 50000
truncated_backprop_length = 15
state_size = 4 # number of layers

number_classes = 2
echo_step = 3
batch_size = 5
number_classes = total_series_length // batch_size // truncated_backprop_length
timestep = 1000

In [26]:
# copying the dataframe so I have a before and after
df_cir_hor_cclk = pd.read_csv('c:/Users/Franz Lom/Documents/TensorFlow/gesture_recognition/Data/extracted_data/cir_hor_cclk.csv') # saving circle horizontal counter clockwise csv as df
df_v_shape = pd.read_csv('c:/Users/Franz Lom/Documents/TensorFlow/gesture_recognition/Data/extracted_data/v_shape.csv')
#df_cir_hor_cclk['shape'] = 1
#df_v_shape['shape'] = 2

df_cir = df_cir_hor_cclk.copy()
df_v = df_v_shape.copy()

df_cir['shape'] = 1
df_v['shape'] = 2


In [52]:
#print(list(df_cir))
df_v.shape

(20045, 24)

In [28]:
x = np.ones([280,1], np.int32)
zero_m = np.zeros([280, 1], np.int32)

circle = np.concatenate((x, zero_m), axis=1)
v_shape = np.concatenate((zero_m, x), axis=1)

labels = np.concatenate([np.array(x), np.array(zero_m)])
labels = labels.flatten()
labels[275:285]
labels.size

560

In [30]:
# data dimension
size = 0 # dont know

size_flat = size * size

shape = (size, size)
# (x, y z)
num_channels = 3

num_classes = 2



In [53]:
def group_drawn_shape(grouped):
    source = [x for x in grouped]
    list_vec = list()
    counter = 0
    padding = np.zeros((500, 3)) # to make all the arrays have a height of 500 and filling with 0
    for v in source:
        df = v[1]
        df = df[['posX', 'posY', 'posZ']]
        x = df.values
        #padding[:x.shape[0], :x.shape[1]] = x
        #print(padding.shape)
        #print()
        #print(padding)
        #x = padding.flatten()
        list_vec.append(x)
        
    
    return np.array(list_vec)

In [54]:
cir_something = group_drawn_shape(df_cir.groupby(['trial', 'tester']))
v_something = group_drawn_shape(df_v.groupby(['trial', 'tester']))

data = np.append(cir_something, v_something ,axis=0)
cir_something.shape

(280,)

In [56]:
#dataStructure = np.concatenate(labels, data)
print('labels: ', labels.shape)
print('data: ', data.shape)

#e = data.reshape(-1, 3)
#print(e.shape)
#data = e

labels:  (560,)
data:  (560,)


In [57]:
x_train, x_test, y_train, y_test  = train_test_split( data, labels, test_size=0.20, random_state=42)


In [58]:
x_train[0][500:550]

array([], shape=(0, 3), dtype=float64)

In [59]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    num_epochs=None,
    shuffle=True
)


In [60]:
#this returns a function
train_input_fn


<function tensorflow.python.estimator.inputs.numpy_io.numpy_input_fn.<locals>.input_fn>

In [61]:
# Calling this function returns a tuple with TensorFlow ops for returning the input and output data:
train_input_fn()


({'x': <tf.Tensor 'random_shuffle_queue_DequeueMany_2:1' shape=(128,) dtype=string>},
 <tf.Tensor 'random_shuffle_queue_DequeueMany_2:2' shape=(128,) dtype=int32>)

In [62]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"x": x_test},
    y=y_test,
    num_epochs= 1,
    shuffle=False
)


In [63]:
# if you want to predict a new set of gestures, we need some input function
some_gesture = x_test[0:5]

In [64]:
# notice how we didnt input an y value as that is what we are predicting
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": some_gesture},
    shuffle=False
)


In [65]:
# We still need y value to  see if the prediction is correct
some_gesture_class = y_test[0:5]


In [66]:
#pre made or canned estimator

features_x = tf.feature_column.numeric_column("x", shape=(2,))
# several input features which then be combined to a list
features_columns = [features_x]
# 3 layer DNN 
num_hidden_units = [512, 256, 128]


In [67]:
model = tf.estimator.DNNClassifier(
    feature_columns=features_columns,
    hidden_units=num_hidden_units,
    activation_fn=tf.nn.relu,
    n_classes=num_classes,
    model_dir="./estimator_checkpoint/"
)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_num_worker_replicas': 1, '_device_fn': None, '_evaluation_master': '', '_keep_checkpoint_max': 5, '_session_config': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_is_chief': True, '_task_id': 0, '_task_type': 'worker', '_log_step_count_steps': 100, '_service': None, '_tf_random_seed': None, '_train_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002541B0DF0F0>, '_model_dir': './estimator_checkpoint/', '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_global_id_in_cluster': 0, '_master': ''}


In [68]:
#training
model.train(input_fn=train_input_fn, steps=2000)


INFO:tensorflow:Calling model_fn.


ValueError: Cannot reshape a tensor with 128 elements to shape [128,2] (256 elements) for 'dnn/input_from_feature_columns/input_layer/x/Reshape' (op: 'Reshape') with input shapes: [128,1], [2] and with input tensors computed as partial shapes: input[1] = [128,2].

In [ ]:
#evaluation
result = model.evaluate(input_fn=test_input_fn)

result


INFO:tensorflow:Calling model_fn.


ValueError: Cannot reshape a tensor with 192000 elements to shape [128,2] (256 elements) for 'dnn/input_from_feature_columns/input_layer/x/Reshape' (op: 'Reshape') with input shapes: [128,1500], [2] and with input tensors computed as partial shapes: input[1] = [128,2].

In [ ]:

#define input functions
def train_input(features, labels, batch_size):
    """An input function for training"""
    # Convert the input to a Dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    
    return dataset.make_one_shot_iterator().get_next()
    
    
    


INFO:tensorflow:Calling model_fn.


ValueError: Cannot reshape a tensor with 192000 elements to shape [128,2] (256 elements) for 'dnn/input_from_feature_columns/input_layer/x/Reshape' (op: 'Reshape') with input shapes: [128,1500], [2] and with input tensors computed as partial shapes: input[1] = [128,2].

In [16]:
#cir_features, c_labels = (group_drawn_shape(df_cir.groupby(['trial', 'tester']), 'circle'))
#v_shape_features, v_labels = (group_drawn_shape(df_v_shape.groupby(['trial', 'tester']), 'v'))
#ink, length, targets = _get_input_tensors(list_of_vec, 'circ')

cir_something = group_drawn_shape(df_cir.groupby(['trial', 'tester']), 'circle' )
v_something = gr

(141, 3)

In [17]:
cirSomething = [['circle', x] for x in cir_something]


IndexError: list index out of range

In [22]:

train_input(cirSomething, 'circle',batch_size)

IndexError: list index out of range

In [27]:
def my_model_fn(features, labels, mode, params):
    """DNN with three hidden layers, and dropout of 0.1 probability."""
    # Create three fully connected layers each layer having a dropout
    # probability of 0.1.
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

    # Compute logits (1 per class).
    logits = tf.layers.dense(net, params['n_classes'], activation=None)

    # Compute predictions.
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Compute loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op')
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)

    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN

    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [365]:

def _get_input_tensors(features, labels):
    """Converts the input dict into inks, lengths, and labels tensors."""
    # features[ink] is a sparse tensor that is [8, batch_maxlen, 3]
    # inks will be a dense tensor of [8, maxlen, 3]
    # shapes is [batchsize,7]
    #shapes = [x for x in features]
    shapes = features[0]
    # lengths will be [batch_size]
    lengths = tf.squeeze(
        tf.slice(shapes, begin=[0, 0], size=[batch_size, 1]))
    inks = tf.reshape(features["ink"], [batch_size, -1, 3])
    if labels is not None:
      labels = tf.squeeze(labels)
    return inks, lengths, labels

In [30]:
my_feature_columns = []
for k, v in cirSomething:
    #print('k: ', k)
    #print('v: ', v)
    
    my_feature_columns.append(tf.feature_column.numeric_column(key=k))


classifier = tf.estimator.Estimator(
    model_fn=my_model_fn,
    params={
        'feature_columns': my_feature_columns,
        # Two hidden layers of 10 nodes each.
        'hidden_units': [10, 10],
        # The model must choose between 3 classes.
        'n_classes': 3,
    })


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\FRANZL~1\\AppData\\Local\\Temp\\tmpd9_55jms', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025218B93DD8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [31]:
classifier.train(
        input_fn=lambda:iris_data.train_input_fn(train_x, train_y, args.batch_size),
        steps=args.train_steps)

NameError: name 'args' is not defined

In [142]:
type(dfc)
#dfc.keys[0]
#print(df_cir.head(100))
#print(df_cir)
#print(dfc.apply)
print(dfc)

{(1, 'B1', 1): Int64Index([14569, 14570, 14571, 14572, 14573, 14574, 14575, 14576, 14577, 14578, 14579, 14580, 14581, 14582, 14583, 14584, 14585, 14586, 14587, 14588, 14589, 14590, 14591, 14592, 14593, 14594, 14595, 14596, 14597, 14598, 14599, 14600, 14601, 14602, 14603, 14604, 14605, 14606, 14607, 14608, 14609, 14610, 14611, 14612, 14613, 14614, 14615, 14616, 14617, 14618, 14619, 14620, 14621, 14622, 14623, 14624, 14625, 14626, 14627, 14628, 14629, 14630, 14631, 14632, 14633, 14634, 14635, 14636, 14637,
            14638, 14639, 14640, 14641, 14642, 14643, 14644, 14645, 14646, 14647, 14648, 14649, 14650, 14651, 14652, 14653, 14654, 14655, 14656, 14657, 14658, 14659],
           dtype='int64'), (1, 'B2', 1): Int64Index([22974, 22975, 22976, 22977, 22978, 22979, 22980, 22981, 22982, 22983,
            ...
            23078, 23079, 23080, 23081, 23082, 23083, 23084, 23085, 23086, 23087], dtype='int64', length=114), (1, 'C1', 1): Int64Index([564, 565, 566, 567, 568, 569, 570, 571, 572, 57

In [133]:
for k, v in dfc.apply.items():
    print('\n', k, '\n')
    print((v))
    break

AttributeError: 'function' object has no attribute 'items'

In [118]:
shapes = features["shape"]
lengths = tf.squeeze(
    tf.slice(shapes, begin=[0, 0], size=[params["batch_size"], 1]))
inks = tf.reshape(
    tf.sparse_tensor_to_dense(features["ink"]),
    [params["batch_size"], -1, 3])
if targets is not None:
  targets = tf.squeeze(targets)

In [157]:
from decimal import Decimal
from base64 import b64encode, b64decode
from json import dumps, loads, JSONEncoder
import pickle

class PythonObjectEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (list, dict, str, int, float, bool, type(None))):
            return super().default(obj)
        return {'_python_object': b64encode(pickle.dumps(obj)).decode('utf-8')}

def as_python_object(dct):
    if '_python_object' in dct:
        return pickle.loads(b64decode(dct['_python_object'].encode('utf-8')))
    return dct

In [152]:
grouped1 = {str(k): v for k, v in grouped1.items()}
grouped2 = {str(k): v for k, v in grouped2.items()}


In [158]:
grouped1 = json.dumps(grouped1, cls=PythonObjectEncoder)
#grouped1 = loads(grouped1, object_hook=as_python_object)

grouped2 = json.dumps(grouped2, cls=PythonObjectEncoder)
#grouped2 = loads(grouped2, object_hook=as_python_object)

In [154]:
with open('./Data/gr1.json', 'w') as outfile1:
    json.dump(grouped1, outfile1, ensure_ascii=False) 

In [155]:
def _get_input_tensors(features, targets):
    shapes = features["shape"]
    lengths = tf.squeeze(
        tf.slice(shapes, begin=[0, 0], size=[params["batch_size"], 1]))
    inks = tf.reshape(
        tf.sparse_tensor_to_dense(features["ink"]),
        [params["batch_size"], -1, 3])
    if targets is not None:
      targets = tf.squeeze(targets)
    

In [156]:
inks, lengths, targets = _get_input_tensors(, targets)
convolved = _add_conv_layers(inks)
final_state = _add_rnn_layers(convolved, lengths)
logits =_add_fc_layers(final_state)

SyntaxError: invalid syntax (<ipython-input-156-866fa2eaead7>, line 1)